In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3,5"
import numpy as np
import keras
from keras.models import model_from_json
from __future__ import division, print_function

Using TensorFlow backend.


In [2]:
from keras import backend as K

list_of_models = ['wide_spi1']

def load_model(model_prefix):
    model_json_string = open(model_prefix+'.json').read()
    model = model_from_json(model_json_string)
    model.load_weights(model_prefix+'.h5')
    return model

In [3]:
model = load_model(list_of_models[0])

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


In [4]:
from keras.models import Sequential
from keras.layers import Conv1D

def reparameterize (model):

    # figure out old conv layers
    config1 = model.layers[0].get_config()
    W1 = model.layers[0].get_weights()[0]
    config2 = model.layers[1].get_config()
    W2 = model.layers[1].get_weights()[0]

    filters = config2['filters']
    kernel_size = config1['kernel_size'][0] + config2['kernel_size'][0] - 1
    padding = config2['padding']
    activation = config2['activation']
    strides = config2['strides']
    input_shape = config1['batch_input_shape'][1:]
    
    # set up new conv layer
    new_model = Sequential()
    new_model.add(Conv1D(input_shape=input_shape,
                         filters=filters,
                         kernel_size=kernel_size,
                         padding=padding,
                         activation=activation,
                         strides=strides))
    
    # get weights for new conv layer
    l1, c1, f1 = W1.shape
    l2, c2, f2 = W2.shape
    new_l, new_c, new_f = new_model.layers[0].get_weights()[0].shape
    
    new_W = np.zeros((new_l, new_c, new_f))
    for k in xrange(0, new_l):
        for c in xrange(0, new_c):
            for f in xrange(0, new_f):
                w = 0
                lb = max(0, k-(l1-1))
                ub = min(k+1, l2)
                for j in xrange(lb, ub):
                    for cc in xrange(0, f1):
                        w = w + (W1[k-j][c][cc]*W2[j][cc][f])
                new_W[k][c][f] = w
    
    # get bias for new conv layer
    new_b = model.layers[1].get_weights()[1]
    
    # set weight and bias
    new_model.layers[0].set_weights((new_W, new_b))
    
    # copy over rest of model
    for layer in model.layers[2:]:
        new_model.add(layer)

    return new_model

In [5]:
new_model = reparameterize (model)

In [6]:
new_model.layers

In [7]:
# small test

from momma_dragonn.data_loaders.hdf5_data_loader import MultimodalAtOnceDataLoader
data = MultimodalAtOnceDataLoader('test_data.hdf5')
test_set = np.array(data.X['sequence'][:50])
test_labels = np.array(data.Y['output'][:50])

model_preds = model.predict(test_set)
new_model_preds = new_model.predict(test_set)

tol = 0.00001
fail = False
for i, e in np.ndenumerate(model_preds):
    if abs(e - new_model_preds[i]) > tol:
        fail = True
        print("FAILED: case ", i[0], e, new_model_preds[i])
if(fail == False):
    print("SUCCESS!")

Input modes [u'sequence']
Output modes [u'output']
FAILED: case  3 0.35747755 0.3470106
FAILED: case  5 0.2940397 0.29324406
FAILED: case  8 0.21484488 0.21513245
FAILED: case  12 0.31092596 0.30761045
FAILED: case  14 0.5050754 0.50520927
FAILED: case  20 0.25404063 0.25903574
FAILED: case  23 0.31186968 0.30043527
FAILED: case  26 0.17476869 0.17211443
FAILED: case  27 0.09961008 0.099342875
FAILED: case  29 0.26790643 0.26186612
FAILED: case  30 0.64360595 0.62310344
FAILED: case  31 0.22456479 0.21259725
FAILED: case  33 0.14903317 0.14518899
FAILED: case  38 0.15994032 0.15906829
FAILED: case  40 0.2818003 0.2806327
FAILED: case  43 0.8915809 0.88913226
FAILED: case  45 0.6767659 0.6717169
FAILED: case  48 0.10540187 0.10089134
FAILED: case  49 0.09348091 0.08839944
